In [8]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import geocoder
import reverse_geocoder as rg
from math import sin, cos, sqrt, atan2, radians

In [2]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
train['isTrain'] = True
test['isTrain'] = False
data = train.append(test, sort=False)

In [24]:
def get_coords(x, y):
    if pd.isnull(x) | pd.isnull(y):
        return 'nan'
    else:
        coordinates = (x, y)
        results = rg.search(coordinates)
        return results

In [91]:
z = data.copy()

In [25]:
z['test'] = z.apply(lambda x: get_coords(x.lat, x.long), axis=1)
z.to_csv('new_data.csv')

In [93]:
new_df = pd.read_csv('new_data.csv')
df = new_df.copy()

In [96]:
df = df.reset_index().drop('index', 1)
df['city_lat'] = df[~df.address_rus.isnull()].test.apply(lambda x: x.split(',')[1])
df['city_long'] = df[~df.address_rus.isnull()].test.apply(lambda x: x.split(',')[3])
df['geo_city'] = df[~df.address_rus.isnull()].test.apply(lambda x: x.split(',')[5])
df['region'] = df[~df.address_rus.isnull()].test.apply(lambda x: x.split(',')[7])
df['city_lat'] = df['city_lat'].astype(str).apply(lambda x: x.replace("'", ''))
df['city_lat'] = df['city_lat'].astype(str).apply(lambda x: x.replace(")", ''))
df['city_long'] = df['city_long'].astype(str).apply(lambda x: x.replace("'", ''))
df['city_long'] = df['city_long'].astype(str).apply(lambda x: x.replace(")", ''))
df['geo_city'] = df['geo_city'].astype(str).apply(lambda x: x.replace("'", ''))
df['geo_city'] = df['geo_city'].astype(str).apply(lambda x: x.replace(")", ''))
df['region'] = df['region'].astype(str).apply(lambda x: x.replace("'", ''))
df['region'] = df['region'].astype(str).apply(lambda x: x.replace(")", ''))
df['city_lat'] = df['city_lat'].astype(float)
df['city_long'] = df['city_long'].astype(float)

In [10]:
R = 6373.0
def distance(x1,y1,x2,y2):    
    dlon = y2 - y1
    dlat = x2 - x1
    a = sin(dlat/2)**2 + cos(x1) * cos(x2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [111]:
df.to_csv('new_data_parsed.csv', index=False)
#df = pd.read_csv('new_data_parsed.csv')

In [16]:
df['diff_from_centre'] = df[~df.address_rus.isnull()].apply(lambda x: distance(x.lat, x.long, x.city_lat, x.city_long), axis=1)
df = df.drop('test', 1)
df.to_csv('data_with_geo.csv')